Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.

Permission is hereby granted, free of charge, to any person obtaining a copy of this
software and associated documentation files (the "Software"), to deal in the Software
without restriction, including without limitation the rights to use, copy, modify,
merge, publish, distribute, sublicense, and/or sell copies of the Software, and to
permit persons to whom the Software is furnished to do so.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED,
INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A
PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT
HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION
OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE
SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

# Welcome to Glue Streaming Interactive Sessions

In this notebook we will familiarize ourselves with interactive development on AWS Glue. Follow along the instructions in each cell to develop, test and deploy an AWS Glue PySpark application. Using this notebook, we will write a streaming consumer that reads data from an Amazon Kinesis Stream **glue-iss-stream-0v8glq** and run some aggregations on the same. The aggregated results are then persisted in an Amazon DynamoDB table **glue-iss-ddbtbl-0v8glq**. <br><br> We will set some initial configurations for the session using predefined magics.  Let us set the idle timeout to 30 minutes, Glue version to 3.0 and the number of workers to 2.  For a complete list of all magics, you can use the **%help** magic.<br><br>The default job type in a Glue interactve session is **glueetl**. We change that to streaming by specifying the **%streaming** magic. Run the below cell to get started.
<br>


In [ ]:
%idle_timeout 30
%glue_version 3.0
%number_of_workers 2

%streaming

## Validate the Job Type

After running the **%streaming** magic, the Job Type has changed from regular **glueetl** to **Streaming**. This confirms that a streaming interactive session has been initialized and the notebook is ready for developing Glue Streaming applications.<br> 


## Initialize the Session
Our next step is to initialize the session. For that, execute the below cell.
<br>

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue import DynamicFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf, PandasUDFType
import re
import pandas as pd
from datetime import datetime
import json
import boto3
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

## Next Steps 

You will notice that the session has been authenticated using the role that was supplied while provisioning the notebook. The output also shows us the current session id. Once we see the message **Session \<session_id\> has been created** let us proceed with the next steps.

## Consume Streaming Data
In the next cell, we will run the command to fetch data from the Kinesis data stream **glue-iss-stream-0v8glq**. Let us execute the below cell. You **need not** wait for a result as there aren't any print statements in this cell. 
<br>


In [ ]:
account_id = boto3.client("sts").get_caller_identity()["Account"]
region_name=boto3.client('s3').meta.region_name
stream_arn= boto3.client('kinesis').describe_stream(StreamName='glue-iss-stream-0v8glq')["StreamDescription"]["StreamARN"]
kinesis_data = glueContext.create_data_frame.from_options(
    connection_type="kinesis",
    connection_options={
        "typeOfData": "kinesis",
        "streamARN": stream_arn,
        "classification": "grok",
        "startingPosition": "earliest",
        "inferSchema": "true",
        "grokPattern":"%{GREEDYDATA:message}"
    },
    transformation_ctx="kinesis_data",
)

## Explore the data
Glue Streaming Interactive session provides a method ```getSampleStreamingDynamicFrame```, which enhances the interactive development experience. The method provides a snapshot of a stream in a static DynamicFrame. You can use this method with an instance of the Glue Context. It takes 3 arguments 
<br>
<br>
1 - the spark streaming dataframe
<br>
2 - ```options``` map
<br>
3 - a ```writeStreamFunction``` to apply a function to every sampled record
<br>
<br>
Available ```options``` are:
<br>
**windowSize** : this parameter determins how long a streaming query will wait after the previous batch was triggered
<br>
**pollingTimeInMs**: this is the total length of time the method will run. It will fire off at least one micro-batch to obtain sample records from the input stream. The time unit is milli-seconds and the value should be **greater** than the **windowSize**
<br>
**recordPollingLimit** : this is defaulted to 100 and will help you set an upper bound on the number of records that is polled from the stream<br>

In the cell that follows, we will sample the incoming stream, count the number of records and also print the first 10 records. Let us execute the cell. 

In [ ]:
options = {
	"pollingTimeInMs": "20000",
	"windowSize": "5 seconds"
}
sampled_dynamic_frame = glueContext.getSampleStreamingDynamicFrame(kinesis_data, options, None)

count_of_sampled_records = sampled_dynamic_frame.count()

print(count_of_sampled_records)

sampled_dynamic_frame.toDF().show(10,False)
              

## Results 

Wait for about 20 seconds to see the output of the previous cell. It depends on your polling time configured in the ```options``` map. Once the output is printed, we see that the default sample size **100** has been polled into the sample data frame and the first 10 messages has been printed to the output cell. <br>
An example record is of the form <br>
```message='88.53.101.3 1234 user1 [26/May/2022:20:55:15 +0000] "GET /page1.html HTTP/1.1" 506 130 "http://example.com/page2.xml" "Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; RM-1127_16056) AppleWebKit/537.36(KHTML, like Gecko) Chrome/42.0.2311.135 Mobile Safari/537.36 Edge/12.10536"'``` <br>
We have to parse the message to give it a structure. The expected parsed result is as below <br>
```
ip_address = 88.53.101.3
client_id = 1234
user_name = user1
timestamp = 26/May/2022:20:55:15 +0000
http_request = GET /page1.html HTTP/1.1
status_code = 506
bytes_transferred = 130
referer = http://example.com/page2.xml
user_agent = Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; RM-1127_16056) AppleWebKit/537.36(KHTML, like Gecko) Chrome/42.0.2311.135 Mobile Safari/537.36 Edge/12.10536
```

<blockquote>If you see that the sampled dynamic frame is empty, it could be caused by a few reasons:

1- The Streaming source is set to Latest and no new data has been ingested during the sampling period.<br>
2- OR The polling time is not enough to process the records it ingested. Data won't show up unless the whole batch has been processed.
</blockquote>

In the next section, we will create a regular expression based python function to process the incoming records. Lets call the function ```reformat```. We will trigger the function and also print out the first 10 parsed records. 
<br>

<div class="alert alert-block alert-warning">
Note : Python UDFs aren't the recommended way to handle data transformations in a Spark application. We use reformat() below to exemplify troubleshooting. When working with a real-world production application, we recommend using native APIs wherever possible. <br>
</div>


In [ ]:

def reformat(partitionData):
    regex='(.*?) (.*?) (.*?) (\[.*?\]) (\".*?\") (.*?) (.*?) (\".*?\") (.*)'
    updatedData = []
    for row in partitionData:
      data =row[0]
      result = re.search(regex, data)
      ip_address=(str(result.group(1)))
      client_id=(str(result.group(2)))
      user_name=(str(result.group(3)))
      time_stamp=datetime.strptime(str(result.group(4)), '[%d/%b/%Y:%H:%M:%S %z]').strftime("%Y-%m-%d %H:%M:%S")
      http_request=(str(result.group(5)))
      page = http_request.split(' ')[1].replace("/","")
      status_code=(int(result.group(6)))
      bytes_transferred=(str(result.group(7)))
      referer=(str(result.group(8)))
      user_agent=(str(result.group(9)))
      updatedData.append([ip_address,client_id,user_name,time_stamp,http_request,page,status_code,bytes_transferred,referer,user_agent])
      error_line = 1/0
    return iter(updatedData)

parsed_df=sampled_dynamic_frame.toDF().rdd.mapPartitions(reformat).toDF(["ip_address","client_id","user_name","time_stamp","http_request","page","status_code","bytes_transferred","referer","user_agent"])

parsed_df.show(10,False)

### Parsing Failed

Did you notice that the above cell failed? Well that is expected. We deliberately created a division by zero exception in our parser.
Comment or delete the line that says  ```error_line = 1/0``` and  rerun the cell. We see that the data has been parsed and successfully displayed.

### Why did we deliberately create an error?

In case of a regular AWS Glue job, in case of errors, the whole application exits and user have to make code changes and resubmit the application. However, in the case of interactve sessions, your coding context and definitions are fully preserved and the session is still operational. There is no need to bootstrap a new cluster and rerun all the preceding transformation. This allows you to focus on quickly iterating your batch function implementations to obtain desirable outcomes. You are able to correct your error and execute them in a matter of seconds. 

### Application Development

Now that we are confident of our data, we can write the stream processing logic in a micro batch implementation. Let us call it the ```processBatch``` method. 
In this method, we will do the below <br>
1 - pass the streaming dataframe in batches of 1 minute<br>
2 - parse the input stream <br>
3 - filter messages with status code >=500<br>
4 - over a one minute intreval, get the count of failures per unique webpage<br>
5 - persist the above metric to a dynamo db table **glue-iss-ddbtbl-0v8glq**<br>
<br>


In [ ]:
def reformat(partitionData):
    regex='(.*?) (.*?) (.*?) (\[.*?\]) (\".*?\") (.*?) (.*?) (\".*?\") (.*)'
    updatedData = []
    for row in partitionData:
      data =json.loads(row[0])["message"]
      result = re.search(regex, data)
      ip_address=(str(result.group(1)))
      client_id=(str(result.group(2)))
      user_name=(str(result.group(3)))
      time_stamp=datetime.strptime(str(result.group(4)), '[%d/%b/%Y:%H:%M:%S %z]').strftime("%Y-%m-%d %H:%M:%S")
      http_request=(str(result.group(5)))
      page = http_request.split(' ')[1].replace("/","")
      status_code=(int(result.group(6)))
      bytes_transferred=(str(result.group(7)))
      referer=(str(result.group(8)))
      user_agent=(str(result.group(9)))
      updatedData.append([ip_address,client_id,user_name,time_stamp,http_request,page,status_code,bytes_transferred,referer,user_agent])
    return iter(updatedData)

def processBatch(data_frame, batchId):
    if data_frame.count() > 0:
        dyFrame = DynamicFrame.fromDF(
            data_frame, glueContext, "from_data_frame"
        )
        
        parsed_df=data_frame.rdd.mapPartitions(reformat).toDF(["ip_address","client_id","user_name","time_stamp","http_request","page","status_code","bytes_transferred","referer","user_agent"])

        filtered_df = parsed_df.filter("status_code >= 500")

        windowed_frame=filtered_df.withWatermark("time_stamp", "1 minutes").groupBy("page",window("time_stamp", "1 minutes")).count()

        drop_columns=['window','count']

        analyzed_frame = windowed_frame.withColumn("occurrence_start_time",col("window.start")).withColumn("occurrence_count",col("count")).withColumn("ddb_partition",concat(col("occurrence_start_time"),col("page"))).drop(*drop_columns)
        
        analyzed_frame_dyf = DynamicFrame.fromDF(analyzed_frame, glueContext, "analyzed_frame_dyf")
        
        store_dyf=glueContext.write_dynamic_frame_from_options(frame=analyzed_frame_dyf,connection_type="dynamodb",connection_options={"dynamodb.output.tableName": "glue-iss-ddbtbl-0v8glq","dynamodb.throughput.write.percent": "1.0"})
        

checkpoint_location="s3://glue-iss-bucket-0v8glq-"+account_id+"-"+region_name+"/0v8glq/"

glueContext.forEachBatch(
    frame=kinesis_data,
    batch_function=processBatch,
    options={
        "windowSize": "60 seconds",
        "checkpointLocation": checkpoint_location
    },
)

## Deploying the Glue Streaming Application

After we successfully run the previous cell, wait for about 2 minutes and confirm you can see the aggregated results in the DynamoDB table **glue-iss-ddbtbl-0v8glq**. <br>So far, we have been developing and testing our application using the streaming interactive sessions. Now that we are confident of the job, let us proceed to convert this into an AWS Glue Job. We have seen that the majority of code cells are doing exploratory analysis and sampling, and are not required to be a part of the main job. 

A commented code cell that represents the final application is provided to you in the next cell. To create a Glue streaming job using this notebook you have 2 options. 
<br>1 - You can uncomment the below cell and delete all other cells
<br>2 - OR do not use the below cell, but delete just the two cells from the notebook that does sampling/debugging and print statements

Once all unwanted cells are deleted and you have the final application code, click on the **Save** button on top to convert the Jupyter notebook to a regular Glue Streaming Job. After the deployment is completed, you can operate your job as you would do with a regular Glue job.  

In [ ]:
'''
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue import DynamicFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf, PandasUDFType
import re
import pandas as pd
from datetime import datetime
import json
import boto3
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

account_id = boto3.client("sts").get_caller_identity()["Account"]
region_name = boto3.client('s3').meta.region_name
stream_arn = boto3.client('kinesis').describe_stream(StreamName='glue-iss-stream-0v8glq')["StreamDescription"]["StreamARN"]
kinesis_data = glueContext.create_data_frame.from_options(
    connection_type="kinesis",
    connection_options={
        "typeOfData": "kinesis",
        "streamARN": stream_arn,
        "classification": "grok",
        "startingPosition": "earliest",
        "inferSchema": "true",
        "grokPattern":"%{GREEDYDATA:message}"
    },
    transformation_ctx="kinesis_data",
)

def reformat(partitionData):
    regex='(.*?) (.*?) (.*?) (\[.*?\]) (\".*?\") (.*?) (.*?) (\".*?\") (.*)'
    updatedData = []
    for row in partitionData:
      data =json.loads(row[0])["message"]
      result = re.search(regex, data)
      ip_address=(str(result.group(1)))
      client_id=(str(result.group(2)))
      user_name=(str(result.group(3)))
      time_stamp=datetime.strptime(str(result.group(4)), '[%d/%b/%Y:%H:%M:%S %z]').strftime("%Y-%m-%d %H:%M:%S")
      http_request=(str(result.group(5)))
      page = http_request.split(' ')[1].replace("/","")
      status_code=(int(result.group(6)))
      bytes_transferred=(str(result.group(7)))
      referer=(str(result.group(8)))
      user_agent=(str(result.group(9)))
      updatedData.append([ip_address,client_id,user_name,time_stamp,http_request,page,status_code,bytes_transferred,referer,user_agent])
    return iter(updatedData)

def processBatch(data_frame, batchId):
    if data_frame.count() > 0:
        dyFrame = DynamicFrame.fromDF(
            data_frame, glueContext, "from_data_frame"
        )
        
        parsed_df=data_frame.rdd.mapPartitions(reformat).toDF(["ip_address","client_id","user_name","time_stamp","http_request","page","status_code","bytes_transferred","referer","user_agent"])

        filtered_df = parsed_df.filter("status_code >= 500")

        windowed_frame=filtered_df.withWatermark("time_stamp", "1 minutes").groupBy("page",window("time_stamp", "1 minutes")).count()

        drop_columns=['window','count']

        analyzed_frame = windowed_frame.withColumn("occurrence_start_time",col("window.start")).withColumn("occurrence_count",col("count")).withColumn("ddb_partition",concat(col("occurrence_start_time"),col("page"))).drop(*drop_columns)
        
        analyzed_frame_dyf = DynamicFrame.fromDF(analyzed_frame, glueContext, "analyzed_frame_dyf")
        
        store_dyf=glueContext.write_dynamic_frame_from_options(frame=analyzed_frame_dyf,connection_type="dynamodb",connection_options={"dynamodb.output.tableName": "glue-iss-ddbtbl-0v8glq","dynamodb.throughput.write.percent": "1.0"})
        

checkpoint_location="s3://glue-iss-bucket-0v8glq-"+account_id+"-"+region_name+"/0v8glq_/"

glueContext.forEachBatch(
    frame=kinesis_data,
    batch_function=processBatch,
    options={
        "windowSize": "60 seconds",
        "checkpointLocation": checkpoint_location
    },
)
'''


## Cost Saving and Clean Up
We have provided the instructions in the blog to cleanup the resources that have been created by the template. Streaming jobs unlike batch jobs run continuosly and can incur additional charges to your AWS Account. The intent of the notebook is to give you a getting started example and you may choose to stop the streaming job if you deployed and ran it as a part of this notebook. 